In [ ]:
import torch 
import os
import numpy as np
import matplotlib.pyplot as plt
import os, glob
import pydicom
import pylab as pl
import sys
import matplotlib.path as mplPath

from collections import Counter
from collections import defaultdict

%matplotlib qt

In [ ]:
class Folder(object): 
    
    def __init__(self, root, subject_substr=""): 
        
        if "\\" in root: 
            raise Exception("Please replace the backslashes in your path with forwardslashes.")
        self.root = root
        self.subfolders = self.get_subjects(subject_substr)
        
        self.walk_history = []
        self.walk_history.append(self.subfolders)

        self.walk_history_lost = []
        self.walk_history_lost.append([])
        
     
    def get_folders_files(self, path):
        for root, folders, files in os.walk(path):
            break   
        
        return folders, files
        
    def get_subjects(self, subject_substr):         
        subjects, _ = self.get_folders_files(self.root)
        
        return [s for s in subjects if subject_substr in s]

    def down(self, equals=[], contains=[], does_not_contain=[], print_absent=False): 
        if not isinstance(equals, list) or not isinstance(contains, list) or not isinstance(does_not_contain, list):
            raise Exception("Please enter a list with strings instead of a string.")
        
        print(f"\nSubfolder equals {str(equals)},   contains {str(contains)},   does not contain {str(does_not_contain)}")
        subfolders_present = []
        subfolders_absent = []
        
        for subfolder in self.subfolders: 
            path = os.path.join(self.root, subfolder)
            
            folders, files = self.get_folders_files(path)
            
            for string in equals: 
                folders = [f for f in folders if string.lower() == f.lower()]
            
            for string in contains: 
                folders = [f for f in folders if string.lower() in f.lower()]
            
            for string in does_not_contain: 
                folders = [f for f in folders if string.lower() not in f.lower()]
            
            if len(folders) == 0: 
                subfolders_absent.append(subfolder)
                
            elif len(folders) == 1: 
                subfolders_present.append(os.path.join(subfolder, folders[0]))
                
            elif len(folders) > 1: 
                print(f"\n{subfolder} has multiple subfolders with the specified name:")
                print(folders)
                subfolders_present.append(os.path.join(subfolder, folders[0]))
        
        if len(subfolders_present) == 0:
            print("\nNo folders contained the specified subfolders, therefore this step is not executed.")
        
        else:    
            print(f"\n{len(subfolders_present)} of {len(self.subfolders)} folders contained specified subfolder.")
            if print_absent: 
                print("\nFolders without subfolder")
                for s in subfolders_absent:
                    print("\t", s)

            self.walk_history.append(subfolders_present)
            self.walk_history_lost.append(subfolders_absent)
            self.subfolders = subfolders_present
            
            print("\nPath example after step:")
            print("\t", self.subfolders[0])
        
            
    def up(self): 
        if len(self.walk_history) == 1:
            print("You can not go up as you are already at the subject folder")
        
        else: 
            self.subfolders = self.walk_history[-2]
            self.walk_history = self.walk_history[:-1]
            self.walk_history_lost = self.walk_history_lost[:-1]  
            
            print("\nPath example after step:")
            print("\t", self.subfolders[0])
    
    def sample_down(self, i=0): 
        subfolder = self.subfolders[i]
        path = os.path.join(self.root, subfolder)
            
        folders, files = self.get_folders_files(path)
        print("\nCurrent path depth:", subfolder)
        print("Subfolders:")
        for s in folders: 
            print("\t", s)
    
    def current_path(self):
        return self.subfolders[0]
            
    def number_of_files(self, print_dict=False):
        
        dct = defaultdict(list)
        
        for subfolder in self.subfolders: 
            path = os.path.join(self.root, subfolder)
            _, files = self.get_folders_files(path)
            length = len(files)
            
            dct[str(length)].append(subfolder)
        
        for l, f in dct.items():
            print(f"\n{str(len(f))} folders contain {l} files each.")
            
        if print_dict: 
            for key, value in dct.items(): 
                print(f"Folders with {key} files:")
                for s in value: 
                    print("\t", s)
            



# Project class

In [ ]:
class Data(object): 
    def __init__(self, path): 
        self.path = path
        self.files = sorted(os.listdir(self.path))
        self.n_files = len(self.files)
        
    def number_of_files(self):
        return self.n_files
    
    def get_slice(self, i): 
        file = self.files[i] #get the i'th slice
        file_path = os.path.join(self.path, file)
        ds = pydicom.dcmread(file_path) # read complete dicom file
        pix = ds.pixel_array # get the slice
        
        return pix
    
    def get_3d_image(self): 
        slices = [] 
        for i in range(self.n_files): 
            slices.append(self.get_slice(i))
        slices = np.dstack(slices)
        
        return slices
        

class Subject(object): 
    def __init__(self, root, name):
        self.root = root 
        self.name = name
        self.pcmra = None
        self.mask = None

    def save_pcmra(self, subpath):
        path = os.path.join(self.root, self.name, subpath)
        self.pcmra = Data(path)
    
    def save_mask(self, subpath):
        path = os.path.join(self.root, self.name, subpath)
        self.mask = Data(path)
        
    def checklist(self, print_false=True): 
        
        complete = True
        
        if self.pcmra == None: 
            if print_false: 
                print(f"sSubject {self.name}: PCMRA not present.")
            complete = False
            
        elif self.mask == None: 
            if print_false: 
                print(f"Subject {self.name}: Mask not present.")
            complete = False
            
        else: 
            pcmra_shape = self.pcmra.get_3d_image().shape
            mask_shape = self.mask.get_3d_image().shape
            if pcmra_shape != mask_shape: 
                print(f"Subject {self.name}: PCMRA and Mask 3d images do not have the same shape")
                print(f"Shape PRMCA: {pcmra_shape}, shape Mask: {mask_shape}")
                
                feedback = input("Do you want to keep this subject (type 'y' or 'n')")
                if feedback == "y" or feedback == "yes": 
                    pass
                else:
                    complete = False
                    
        return complete
            
        
class Project(object):
    def __init__(self, root): 
        self.root = root
        self.subjects = {}
        
    def add_subject(self, name): 
        self.subjects[name] = Subject(self.root, name)
    
    def remove_subject(self, name): 
        del self.subjects[name]
        
    def get_subjects(self, index=-1):
        return list(self.subjects.keys())[:index]
    
    def remove_incomplete_subjects(self):
        subjects = self.get_subjects()
        complete_subjects = {}
        for sub in subjects: 
            if self.subjects[sub].checklist(): 
                complete_subjects[sub] = self.subjects[sub]
        self.subjects = complete_subjects

## Carotide

In [ ]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/afterfifteen/data/recon"
subjects_dir = root + path

carotide = Folder(subjects_dir)

dictionary = {"root": root}
dictionary["carotide"] = {"path": path + "/"} 

In [ ]:
c_proj = Project(subjects_dir)
print(c_proj.root)

In [ ]:
carotide.down(contains=["wip4dflow"])

In [ ]:
carotide.down(contains=["dicom"])

In [ ]:
carotide.down(equals=["pc_mra"])

In [ ]:
for string in sorted(carotide.subfolders):
    subj, loc = string.split("/", 1)
    c_proj.add_subject(subj)
    c_proj.subjects[subj].save_pcmra(loc)

In [ ]:
subjects = c_proj.get_subjects(5)
print(subjects)

In [ ]:
im = c_proj.subjects["AF_0001"].pcmra.get_3d_image()
print(im.shape)

In [ ]:
carotide.up()
carotide.down(contains=["mask_left"])
carotide.number_of_files()

In [ ]:
for string in sorted(carotide.subfolders):
    subj, loc = string.split("/", 1)
    c_proj.subjects[subj].save_mask(loc)

In [ ]:
c_proj.remove_incomplete_subjects()
print(c_proj.get_subjects())

In [ ]:
subjects = 

## Aorta Volunteers

In [ ]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/4dflow/4D-flow Volunteers/Aorta"
subjects_dir = root + path

aorta_v = Folder(subjects_dir)

In [ ]:
aorta_v.down(contains=["_pca"], print_absent=True)

In [ ]:
aorta_v.down(equals=["dicom"])

#### Waarom hebben 2 van onderstaande volunteers meer slices in de 3d opname? Zijn de dimensies per slice hier anders?

In [ ]:
# aorta_v.up()
aorta_v.down(equals=["mask"])
aorta_v.number_of_files()

In [ ]:
aorta_v.up()
aorta_v.down(equals=["M"])
aorta_v.number_of_files()

In [ ]:
aorta_v.up()
aorta_v.up()
aorta_v.up()
aorta_v.down(contains=["_sense"], does_not_contain=["dont_use"])

In [ ]:
aorta_v.down(contains=["dicom"])
# aorta_v.sample_down()

In [ ]:
aorta_v.down(contains=["pc_mra"])
aorta_v.number_of_files()

## Aorta BaV

In [ ]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/4dflow/4D-flow BAV"
subjects_dir = root + path
print(subjects_dir)

aorta_b = Folder(subjects_dir, subject_substr="done")

In [ ]:
aorta_b.down(contains=["_PCA"]) 

In [ ]:
aorta_b.down(contains=["dicom"])

#### aorta_122_withgap heeft een eerste slice met segmentatie, die in Aorta zelf mist. Bij 121 is er ook enkel een verschil in de eerste slice.

In [ ]:
aorta_b.down(contains=["aorta"],print_absent=True)

In [ ]:
aorta_b.up()
aorta_b.down(equals=["Aorta"])

#### waarom verschilt het aantal slices? Is de dimensie per slice wel gelijk?

In [ ]:
aorta_b.number_of_files()

In [ ]:
aorta_b.up()
aorta_b.down(equals=["M"], print_absent=True)

#### hier komt het aantal wel overeen met pc_mra. Dus aantal tijdsstappen is hetzelfde

In [ ]:
aorta_b.number_of_files()

In [ ]:
aorta_b.up()
aorta_b.up()
aorta_b.up()
aorta_b.down(contains=["_sense"])

In [ ]:
aorta_b.down(contains=["dicom"])
# aorta_b.sample_down(3)

In [ ]:
aorta_b.down(contains=["pc_mra"])

In [ ]:
aorta_b.number_of_files()

## Aorta CoA

In [ ]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/4dflow/4D-flow CoA"
subjects_dir = root + path
print(subjects_dir)

aorta_c = Folder(subjects_dir, subject_substr="done")
# aorta_c.sample_down(2)

In [ ]:
aorta_c.down(contains=["_pca"])
# aorta_c.sample_down()

In [ ]:
aorta_c.down(contains=["dicom"])

#### weer meerdere segmentaties, hoe behandel ik deze?

In [ ]:
aorta_c.down(contains=["aorta"])

#### verschil in aantal slices

In [ ]:
aorta_c.number_of_files()

In [ ]:
aorta_c.up()
aorta_c.down(equals=["M"])

In [ ]:
aorta_c.number_of_files()

In [ ]:
aorta_c.up()
aorta_c.up()
aorta_c.up()

In [ ]:
aorta_c.down(contains=["_sense"])

In [ ]:
aorta_c.down(contains=["dicom"])

In [ ]:
aorta_c.down(contains=["pc_mra"])

In [ ]:
aorta_c.number_of_files()

## Aorta RESV

In [ ]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/resvcue_marfan/analysis/RESVcue Marfan"
subjects_dir = root + path
print(subjects_dir)

aorta_r = Folder(subjects_dir, subject_substr="RESV")

In [ ]:
aorta_r.down()

In [ ]:
aorta_r.down(contains=["segmentatie"], print_absent=True)

#### weer verschil in aantal slices. Zijn dimensies hetzelfde?

In [ ]:
aorta_r.number_of_files()

L:\basic\divi\Projects\resvcue_marfan\analysis\RESVcue Marfan\RESV_001\V1 21-10-2019\4D flow recon out\re_21102019_1526-out\DICOM_PCMRA
L:\basic\divi\Projects\resvcue_marfan\analysis\RESVcue Marfan\RESV_001\V1 21-10-2019\Segmentatie

L:\basic\divi\Projects\4dflowimpulse\Pilot study with LUMC\Scans\20181029_2_EPI-PROUD\4D flow\recon_out_20181029_2_PROUD_noNC\PCMRA
L:\basic\divi\Projects\4dflowimpulse\Pilot study with LUMC\Scans\20181029_2_EPI-PROUD\SEGM\PROUD SEGM

L:\basic\divi\Projects\cs_4dflow\project_VitalEyeComparisonClinical\data\subject_01\recon_out_subject_01_VE\PCMRA
L:\basic\divi\Projects\cs_4dflow\project_VitalEyeComparisonClinical\data\subject_01\segm_sub01



In [ ]:
aorta_r.up()

In [ ]:
aorta_r.down(contains=["4d flow"], print_absent=True)

In [ ]:
aorta_r.down(print_absent=True)

In [ ]:
aorta_r.subfolders

In [ ]:
aorta_r.down(contains=["dicom_pcmra"], print_absent=True)

In [ ]:
aorta_r.down(contains=["pcmra"], print_absent=True)

#### Dit waren de getallen voor segmentatie:

15 folders contain 30 files each.

40 folders contain 24 files each.

2 folders contain 0 files each.

#### Komst niet overeen met de PCMRA, wat gebeurt er.

In [ ]:
aorta_r.number_of_files()

## Heart LUMC

In [ ]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/4dflowimpulse/Pilot study with LUMC/Scans"
subjects_dir = root + path
print(subjects_dir)

heart_l = Folder(subjects_dir, subject_substr="PROUD")

In [ ]:
heart_l.down(contains=["segm"])

In [ ]:
heart_l.down(contains=['proud'], print_absent=True)

In [ ]:
heart_l.number_of_files()

In [ ]:
heart_l.up()
heart_l.up()

In [ ]:
heart_l.down(contains=['4d flow'], print_absent=True)

In [ ]:
heart_l.down(contains=["proud"])

In [ ]:
# heart_l.subfolders

#### Wat is het verschil tussen deze twee?

In [ ]:
heart_l.down(contains=["pcmra"])

In [ ]:
heart_l.down()

In [ ]:
heart_l.number_of_files()

## Heart Vital

In [ ]:
root = "/home/ptenkaate/lood_storage/"
path = "divi/Projects/cs_4dflow/project_VitalEyeComparisonClinical/data"
subjects_dir = root + path
print(subjects_dir)

heart_v = Folder(subjects_dir)
heart_v.sample_down(2)

In [ ]:
heart_v.down(contains=["segm"])
heart_v.subfolders

In [ ]:
heart_v.up()

In [ ]:
heart_v.down(contains=["VE"])


In [ ]:
heart_v.down(contains=["pcmra"])


In [ ]:
heart_v.number_of_files()